# Lesson 7: Build a Live Camera App

Rita Philavanh

20170905

## Retrieve Street View House Number (SVHN) training/test dataset

Download train.tar.gz and test.tar.gz files from http://ufldl.stanford.edu/housenumbers/ 

## Pre-processing

### Extract dataset from compressed tar.gz file

In [2]:
import os
import sys
import tarfile
import numpy as np

num_classes = 0
np.random.seed(133)
data_root = '.'

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract('train.tar.gz')
test_folders = maybe_extract('test.tar.gz')

train already present - Skipping extraction of train.tar.gz.
[]
test already present - Skipping extraction of test.tar.gz.
[]


### Extract labels from .mat file 

(used digiStruct.py code found here: https://github.com/prijip/Py-Gsvhn-DigitStruct-Reader/blob/360a03909a22a28e2e814e83d6915ee5f915e5da/digitStruct.py)



In [258]:
#import h5py
#import numpy as np
#digitStruct = h5py.File('train/digitStruct.mat')
#digitStruct.keys()
#mat = {}
#for k, v in digitStruct.items():
#    print(k,v)
#    mat[k] = np.array(v)

#print(mat)


import h5py
import numpy as np

#
# Bounding Box
#
class BBox:
    def __init__(self):
        self.label = ""     # Digit
        self.left = 0
        self.top = 0
        self.width = 0
        self.height = 0

class DigitStruct:
    def __init__(self):
        self.name = None    # Image file name
        self.bboxList = None # List of BBox structs

# Function for debugging
def printHDFObj(theObj, theObjName):
    isFile = isinstance(theObj, h5py.File)
    isGroup = isinstance(theObj, h5py.Group)
    isDataSet = isinstance(theObj, h5py.Dataset)
    isReference = isinstance(theObj, h5py.Reference)
    print("{}".format(theObjName))
    print("    type(): {}".format(type(theObj)))
    if isFile or isGroup or isDataSet:
        # if theObj.name != None:
        #    print "    name: {}".format(theObj.name)
        print("    id: {}".format(theObj.id))
    if isFile or isGroup:
        print("    keys: {}".format(theObj.keys()))
    if not isReference:
        print("    Len: {}".format(len(theObj)))

    if not (isFile or isGroup or isDataSet or isReference):
        print(theObj)

def readDigitStructGroup(dsFile):
    dsGroup = dsFile["digitStruct"]
    return dsGroup

#
# Reads a string from the file using its reference
#
def readString(strRef, dsFile):
    strObj = dsFile[strRef]
    str = ''.join(chr(i) for i in strObj)
    return str

#
# Reads an integer value from the file
#
def readInt(intArray, dsFile):
    intRef = intArray[0]
    isReference = isinstance(intRef, h5py.Reference)
    intVal = 0
    if isReference:
        intObj = dsFile[intRef]
        intVal = int(intObj[0])
    else: # Assuming value type
        intVal = int(intRef)
    return intVal

def yieldNextInt(intDataset, dsFile):
    for intData in intDataset:
        intVal = readInt(intData, dsFile)
        yield intVal 

def yieldNextBBox(bboxDataset, dsFile):
    for bboxArray in bboxDataset:
        bboxGroupRef = bboxArray[0]
        bboxGroup = dsFile[bboxGroupRef]
        labelDataset = bboxGroup["label"]
        leftDataset = bboxGroup["left"]
        topDataset = bboxGroup["top"]
        widthDataset = bboxGroup["width"]
        heightDataset = bboxGroup["height"]

        left = yieldNextInt(leftDataset, dsFile)
        top = yieldNextInt(topDataset, dsFile)
        width = yieldNextInt(widthDataset, dsFile)
        height = yieldNextInt(heightDataset, dsFile)

        bboxList = []

        for label in yieldNextInt(labelDataset, dsFile):
            bbox = BBox()
            bbox.label = label
            bbox.left = next(left)
            bbox.top = next(top)
            bbox.width = next(width)
            bbox.height = next(height)
            bboxList.append(bbox)

        yield bboxList

def yieldNextFileName(nameDataset, dsFile):
    for nameArray in nameDataset:
        nameRef = nameArray[0]
        name = readString(nameRef, dsFile)
        yield name

# dsFile = h5py.File('train/digitStruct.mat', 'r')
def yieldNextDigitStruct(dsFileName):
    dsFile = h5py.File(dsFileName, 'r')
    dsGroup = readDigitStructGroup(dsFile)
    nameDataset = dsGroup["name"]
    bboxDataset = dsGroup["bbox"]

    bboxListIter = yieldNextBBox(bboxDataset, dsFile)
    for name in yieldNextFileName(nameDataset, dsFile):
        bboxList = next(bboxListIter)
        obj = DigitStruct()
        obj.name = name
        obj.bboxList = bboxList
        yield obj

L = []
y1 = []
y2 = []
y3 = []
y4 = []
y5 = []
def testMain():
    i = 0
    dsFileName = 'train/digitStruct.mat'
    testCounter = 0

    for dsObj in yieldNextDigitStruct(dsFileName):
        # testCounter += 1
        print(dsObj.name)
        print(len(dsObj.bboxList))
        L.append(len(dsObj.bboxList))
        
        cnty = 0
        for bbox in dsObj.bboxList:
            print("    {}:{},{},{},{}".format(
                bbox.label, bbox.left, bbox.top, bbox.width, bbox.height))
            
            if cnty == 0: y1.append(bbox.label)
            elif cnty == 1: y2.append(bbox.label) 
            elif cnty == 2: y3.append(bbox.label)
            elif cnty == 3: y4.append(bbox.label)
            elif cnty == 4: y5.append(bbox.label)
            
            cnty += 1
            
        if cnty < 5:
            if len(dsObj.bboxList) < 1: y1.append(10)
            if len(dsObj.bboxList) < 2: y2.append(10) 
            if len(dsObj.bboxList) < 3: y3.append(10)
            if len(dsObj.bboxList) < 4: y4.append(10)
            if len(dsObj.bboxList) < 5: y5.append(10)
        if testCounter >= 5:
            break
        i +=1
        
if __name__ == "__main__":
    testMain()

1.png
2
    1:246,77,81,219
    9:323,81,96,219
2.png
2
    2:77,29,23,32
    3:98,25,26,32
3.png
2
    2:17,5,8,15
    5:25,5,9,15
4.png
2
    9:57,13,15,34
    3:72,13,13,34
5.png
2
    3:52,7,21,46
    1:74,10,15,46
6.png
2
    3:28,6,10,21
    3:38,8,11,21
7.png
2
    2:35,10,13,32
    8:47,11,13,32
8.png
3
    7:17,4,7,15
    4:25,4,6,15
    4:31,3,7,15
9.png
3
    1:19,4,14,24
    2:29,4,13,24
    8:38,5,17,24
10.png
2
    1:25,6,9,27
    6:34,4,14,27
11.png
2
    2:22,5,12,26
    3:33,4,9,26
12.png
2
    6:67,16,23,55
    3:90,17,20,55
13.png
2
    4:44,12,17,33
    2:63,13,18,33
14.png
2
    5:35,2,21,42
    8:55,2,23,42
15.png
2
    1:22,8,14,20
    6:33,7,10,20
16.png
2
    2:20,3,7,19
    3:26,1,7,19
17.png
2
    7:23,4,14,22
    9:35,4,12,22
18.png
2
    5:52,12,28,41
    3:80,17,21,41
19.png
3
    2:19,1,7,19
    2:28,1,7,19
    2:36,1,7,19
20.png
2
    6:20,2,11,24
    2:32,1,11,24
21.png
1
    2:72,6,52,85
22.png
3
    5:24,4,10,16
    1:34,5,7,16
    5:40,5,11,16
23.png

255.png
2
    9:55,8,34,55
    3:82,8,37,55
256.png
2
    2:31,2,20,33
    8:50,3,18,33
257.png
2
    1:80,30,17,41
    1:103,35,15,41
258.png
2
    5:45,1,34,60
    6:78,3,33,60
259.png
1
    5:51,14,19,30
260.png
2
    3:2,2,13,20
    6:18,2,17,20
261.png
2
    1:15,1,4,16
    4:18,1,6,16
262.png
3
    2:61,23,18,44
    4:78,25,23,44
    1:59,26,20,46
263.png
2
    7:46,14,15,38
    1:62,14,10,38
264.png
4
    2:27,5,16,28
    1:44,6,11,28
    9:57,3,12,28
    10:67,5,16,28
265.png
2
    7:46,10,25,45
    6:70,7,25,45
266.png
2
    5:125,27,54,89
    10:173,26,50,89
267.png
2
    2:3,9,20,54
    1:33,11,18,54
268.png
1
    6:34,12,8,24
269.png
3
    3:33,7,16,32
    5:52,5,17,32
    3:68,6,20,32
270.png
3
    2:25,3,12,30
    5:37,2,10,30
    5:51,1,13,30
271.png
2
    1:18,1,8,24
    9:26,1,11,24
272.png
3
    4:44,7,11,35
    6:57,6,8,35
    4:69,4,15,35
273.png
1
    9:31,8,18,27
274.png
3
    1:28,3,9,28
    6:36,3,13,28
    6:47,4,13,28
275.png
2
    2:27,4,16,22
    5:41,4,20,2

    5:26,3,10,21
616.png
1
    7:45,6,30,49
617.png
2
    2:53,18,18,32
    8:73,18,18,32
618.png
3
    3:127,62,24,39
    2:144,55,26,39
    6:170,62,18,39
619.png
1
    3:55,6,38,56
620.png
2
    4:65,16,30,54
    6:97,13,28,54
621.png
2
    6:28,1,20,36
    10:51,2,17,36
622.png
2
    3:21,4,10,23
    3:30,2,9,23
623.png
1
    9:133,37,35,62
624.png
4
    2:40,11,18,27
    1:58,12,14,27
    1:71,11,11,27
    8:81,10,16,27
625.png
2
    2:30,5,14,29
    5:40,8,11,29
626.png
3
    6:70,26,26,45
    5:89,21,33,45
    8:119,12,24,45
627.png
2
    2:32,4,17,35
    4:45,3,18,35
628.png
2
    1:22,1,12,25
    6:37,2,12,25
629.png
3
    2:147,27,11,21
    3:151,24,11,21
    4:158,23,12,21
630.png
1
    8:23,5,11,20
631.png
2
    1:59,13,26,52
    9:81,5,33,52
632.png
1
    6:26,1,16,24
633.png
3
    1:32,12,8,18
    7:39,13,9,18
    4:48,13,11,18
634.png
3
    5:63,19,14,27
    8:81,22,12,27
    8:93,22,26,27
635.png
1
    5:32,2,14,26
636.png
3
    1:13,3,3,11
    1:17,3,3,11
    5:22,2,6,

    2:38,9,14,29
    7:50,9,13,29
855.png
3
    1:33,6,13,31
    10:46,8,11,31
    9:55,8,8,31
856.png
3
    1:43,11,12,30
    7:58,11,11,30
    8:68,7,10,30
857.png
2
    8:58,21,18,31
    3:73,12,21,31
858.png
2
    4:24,6,7,19
    5:32,5,7,19
859.png
2
    2:15,1,5,18
    7:20,1,4,18
860.png
1
    2:31,6,11,29
861.png
2
    5:32,10,8,18
    1:40,12,7,18
862.png
2
    1:24,4,6,20
    5:31,5,9,20
863.png
3
    1:43,6,13,34
    10:55,6,19,34
    8:72,8,18,34
864.png
1
    2:35,2,33,40
865.png
3
    2:31,6,10,25
    9:42,7,9,25
    10:52,8,7,25
866.png
3
    1:74,16,27,58
    6:103,21,29,58
    10:131,24,29,58
867.png
3
    2:1,4,5,15
    6:8,4,5,15
    5:17,4,6,15
868.png
2
    5:21,3,8,23
    10:29,4,7,23
869.png
3
    1:38,12,5,19
    1:43,14,7,19
    9:48,13,8,19
870.png
3
    2:27,7,11,25
    3:37,6,11,25
    1:47,9,10,25
871.png
2
    5:15,2,7,16
    7:20,3,6,16
872.png
1
    6:29,5,12,25
873.png
3
    7:61,12,26,56
    4:81,13,19,56
    5:97,13,27,56
874.png
2
    1:33,4,8,30
   

    2:52,8,21,37
    6:75,8,23,37
1209.png
2
    5:49,11,16,37
    8:65,10,18,37
1210.png
3
    2:21,4,9,21
    6:31,3,6,21
    4:40,6,9,21
1211.png
2
    4:27,2,13,27
    7:39,3,13,27
1212.png
3
    1:62,20,25,48
    2:81,11,17,48
    1:94,7,23,48
1213.png
3
    1:42,8,15,30
    1:57,8,12,30
    4:70,8,15,30
1214.png
3
    3:56,13,13,25
    9:68,11,11,25
    2:81,13,9,25
1215.png
2
    5:32,1,21,34
    8:53,7,21,34
1216.png
2
    4:49,7,25,55
    5:76,6,24,55
1217.png
2
    2:39,17,7,23
    2:47,11,9,23
1218.png
1
    7:22,7,6,14
1219.png
3
    2:42,12,12,32
    8:54,11,10,32
    4:62,10,14,32
1220.png
2
    5:77,22,22,47
    4:101,25,24,47
1221.png
3
    3:39,3,14,40
    2:58,3,15,40
    6:75,6,19,40
1222.png
2
    7:63,15,22,35
    9:86,18,21,35
1223.png
2
    6:42,6,17,39
    4:57,9,12,39
1224.png
2
    2:80,16,36,79
    8:118,7,45,79
1225.png
2
    4:57,23,19,45
    10:69,12,23,45
1226.png
2
    4:57,10,16,55
    7:72,14,21,55
1227.png
2
    3:29,4,22,30
    8:52,5,21,30
1228.png


...

    7:71,2,9,24
32069.png
2
    1:63,14,19,57
    4:82,14,40,57
32070.png
2
    4:38,11,17,26
    4:55,10,14,26
32071.png
2
    3:26,5,22,28
    7:49,7,21,28
32072.png
3
    4:77,39,16,37
    3:95,27,21,37
    1:118,18,18,37
32073.png
1
    7:33,7,16,33
32074.png
2
    4:159,53,38,73
    2:190,74,39,73
32075.png
2
    1:23,3,5,19
    6:28,3,8,19
32076.png
4
    2:25,9,10,16
    5:36,9,10,16
    10:45,9,11,16
    7:57,9,11,16
32077.png
2
    1:52,9,17,47
    10:68,8,19,47
32078.png
3
    1:18,3,6,16
    4:23,4,8,16
    10:30,4,9,16
32079.png
2
    2:29,7,6,24
    8:36,7,10,24
32080.png
2
    1:21,3,9,20
    6:30,3,10,20
32081.png
2
    2:42,3,36,48
    8:73,7,36,48
32082.png
4
    1:38,4,14,29
    1:49,7,11,29
    7:59,5,14,29
    5:73,5,15,29
32083.png
2
    3:19,2,12,23
    4:33,2,10,23
32084.png
1
    3:19,2,9,15
32085.png
2
    1:37,5,8,29
    5:51,4,18,29
32086.png
2
    5:42,12,18,26
    10:55,10,16,26
32087.png
3
    4:80,21,36,59
    4:126,29,29,59
    9:162,32,37,59
32088.png
1

    6:25,2,11,27
    9:35,1,12,27
32311.png
2
    2:50,11,13,30
    4:60,10,12,30
32312.png
2
    9:49,7,20,32
    8:68,12,18,32
32313.png
1
    7:32,8,18,32
32314.png
3
    3:31,7,14,27
    2:43,7,16,27
    7:59,8,17,27
32315.png
2
    3:13,1,8,13
    9:20,1,10,13
32316.png
3
    5:27,6,10,24
    5:37,5,13,24
    9:50,5,13,24
32317.png
3
    1:22,4,5,18
    8:26,3,9,18
    6:34,4,7,18
32318.png
1
    1:56,16,9,33
32319.png
2
    2:42,10,16,32
    5:61,12,13,32
32320.png
2
    5:28,4,12,23
    3:40,4,11,23
32321.png
2
    1:52,11,14,39
    3:66,11,21,39
32322.png
2
    4:18,2,11,16
    7:29,2,11,16
32323.png
3
    1:89,19,23,70
    8:124,17,35,70
    10:165,19,28,70
32324.png
2
    8:113,45,21,42
    6:135,46,19,42
32325.png
3
    2:24,5,5,15
    1:28,5,5,15
    10:32,4,7,15
32326.png
2
    6:17,44,41,74
    8:61,44,32,74
32327.png
3
    1:14,3,12,22
    10:27,2,14,22
    10:41,2,16,22
32328.png
3
    2:15,1,4,15
    10:19,1,3,15
    8:21,1,4,15
32329.png
2
    1:35,9,16,30
    5:51,7,

32672.png
4
    1:21,5,6,19
    8:33,6,9,19
    7:44,6,9,19
    10:56,6,11,19
32673.png
2
    3:25,5,11,25
    2:38,6,13,25
32674.png
2
    1:42,6,26,58
    5:64,5,40,58
32675.png
2
    3:35,8,13,31
    10:46,8,14,31
32676.png
1
    5:31,6,11,26
32677.png
3
    1:59,20,11,24
    9:69,20,13,24
    1:81,18,12,24
32678.png
2
    3:22,2,9,23
    2:30,2,8,23
32679.png
3
    1:65,16,13,34
    8:80,20,14,34
    3:98,14,17,34
32680.png
2
    3:23,3,11,21
    1:32,2,10,21
32681.png
1
    5:50,14,20,41
32682.png
3
    2:26,1,12,29
    6:40,3,14,29
    5:54,4,17,29
32683.png
2
    4:25,4,6,16
    7:31,4,9,16
32684.png
2
    4:81,17,44,77
    10:132,20,47,77
32685.png
3
    1:56,13,13,43
    1:72,11,7,43
    6:83,12,20,43
32686.png
2
    2:44,7,41,48
    7:80,5,30,48
32687.png
3
    7:108,24,11,23
    5:118,25,9,23
    2:126,23,11,23
32688.png
2
    5:184,43,37,45
    9:211,23,34,45
32689.png
1
    10:55,8,14,27
32690.png
2
    7:45,11,15,35
    7:61,10,10,35
32691.png
3
    1:46,6,27,51
    6:71,

2
    1:39,7,11,32
    8:53,6,21,32
32910.png
2
    5:27,5,14,28
    1:41,7,9,28
32911.png
2
    5:51,8,30,61
    3:80,10,32,61
32912.png
3
    2:56,14,20,40
    3:76,17,20,40
    2:93,17,18,40
32913.png
2
    5:36,4,24,39
    1:63,4,16,39
32914.png
1
    9:36,5,10,24
32915.png
2
    8:44,7,24,43
    10:70,9,25,43
32916.png
2
    3:55,12,19,39
    6:74,11,21,39
32917.png
2
    3:32,2,11,31
    6:43,2,13,31
32918.png
2
    2:19,4,8,18
    1:28,4,8,18
32919.png
2
    3:54,13,23,50
    10:76,12,21,50
32920.png
3
    1:32,3,21,45
    5:54,3,18,45
    10:75,4,21,45
32921.png
3
    9:120,39,24,30
    5:144,48,18,30
    7:160,48,17,30
32922.png
2
    9:19,4,10,17
    10:32,4,7,17
32923.png
3
    4:65,19,14,27
    9:76,18,12,27
    6:86,19,15,27
32924.png
2
    3:44,11,27,45
    6:71,8,29,45
32925.png
2
    4:48,8,23,38
    8:74,6,25,38
32926.png
2
    1:42,10,6,22
    9:48,12,7,22
32927.png
2
    1:60,15,13,53
    5:73,12,21,53
32928.png
2
    8:78,22,14,25
    10:93,26,13,25
32929.png
3
    

33142.png
3
    2:57,15,22,37
    6:80,14,20,37
    6:101,15,19,37
33143.png
3
    1:12,1,4,11
    1:15,1,3,11
    3:19,1,6,11
33144.png
2
    6:56,10,30,37
    9:84,14,32,37
33145.png
3
    1:57,15,23,39
    2:86,16,26,39
    8:118,15,25,39
33146.png
2
    8:55,10,15,27
    6:72,6,15,27
33147.png
3
    1:35,5,11,35
    3:48,5,23,35
    6:69,7,18,35
33148.png
3
    1:106,36,31,41
    6:136,35,28,41
    1:172,21,17,41
33149.png
2
    1:70,18,24,62
    10:92,16,37,62
33150.png
2
    5:33,6,14,31
    10:47,8,11,31
33151.png
2
    2:27,1,23,33
    7:49,2,19,33
33152.png
1
    8:20,1,13,23
33153.png
2
    2:62,23,19,36
    6:83,21,18,36
33154.png
2
    2:24,2,16,23
    4:40,4,14,23
33155.png
1
    5:34,4,31,41
33156.png
3
    1:100,37,36,63
    6:122,28,34,63
    4:151,23,24,63
33157.png
3
    1:49,9,14,40
    9:59,5,21,40
    8:80,1,18,40
33158.png
3
    1:24,2,8,26
    4:34,4,6,26
    2:40,3,8,26
33159.png
4
    1:23,4,9,19
    7:33,1,10,19
    10:47,2,7,19
    10:59,4,8,19
33160.png
3
  

### Check Labels

In [259]:
print(len(L))
print(L[:10]) # length of digits
print(y1[:10]) # 1st digit
print(y2[:10]) # 2nd digit
print(y3[:10]) # ...etc (If no digit, then 'None')
print(y4[:10])
print(y5[:10])

33402
[2, 2, 2, 2, 2, 2, 2, 3, 3, 2]
[1, 2, 2, 9, 3, 3, 2, 7, 1, 1]
[9, 3, 5, 3, 1, 3, 8, 4, 2, 6]
[10, 10, 10, 10, 10, 10, 10, 4, 8, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [ ]:
#import scipy.io as sio
#from tflearn.data_utils import shuffle, to_categorical
#
##Dataset location
#train_location = './train_32x32.mat'
#test_location = './test_32x32.mat'
#
#
#def load_train_data():
#    train_dict = sio.loadmat(train_location)
#    X = np.asarray(train_dict['X'])
#    print(type(X), X.shape) #'numpy.ndarray'; 32x32 pixels, 3 RGB, 73257 samples 
#    X_train = []
#    for i in range(X.shape[3]):
#        X_train.append(X[:,:,:,i])
#    X_train = np.asarray(X_train)
#    print(type(X_train), X_train.shape)
#    print(X_train[0][:1])
#    
#    Y_train = train_dict['y']
#    print(Y_train[:8])
#    print(len(Y_train))  73257 samples 
#    for i in range(len(Y_train)):
#        if Y_train[i]%10 == 0:
#            Y_train[i] = 0
#            
#    print(Y_train[:8])
#    print(type(Y_train), Y_train.shape)
#    
#    Y_train = to_categorical(Y_train,10)
#    print(Y_train[:8])
#    print(type(Y_train), Y_train.shape)
#    return (X_train,Y_train)
#
#def load_test_data():
#    test_dict = sio.loadmat(test_location)
#    X = np.asarray(test_dict['X'])
#
#    X_test = []
#    for i in range(X.shape[3]):
#        X_test.append(X[:,:,:,i])
#    X_test = np.asarray(X_test)
#
#    Y_test = test_dict['y']
#    for i in range(len(Y_test)):
#        if Y_test[i]%10 == 0:
#            Y_test[i] = 0
#    Y_test = to_categorical(Y_test,10)
#    return (X_test,Y_test)
#
#X_train, Y_train = load_train_data()


### Convert dataset to 32x32 pixels grayscale matrix

In [261]:
import cv2

X = []


pic_num = 1
#print(len(os.listdir('./train')))
for i in range(len(os.listdir('./train'))): 
    img = cv2.imread("./train/"+str(pic_num) +".png",cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (32, 32))
    X.append(resized_image)
    pic_num +=1

error: C:\projects\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:3483: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize


### Split dataset into train and validation set

In [262]:
X_valid = X[:10000]
X_train = X[10000:]

print(len(X), len(X_valid), len(X_train))

L_valid = L[:10000] 
L_train = L[10000:] 
y1_valid = y1[:10000]
y2_valid = y2[:10000]
y3_valid = y3[:10000]
y4_valid = y4[:10000]
y5_valid = y5[:10000]
y1_train = y1[10000:]
y2_train = y2[10000:]
y3_train = y3[10000:]
y4_train = y4[10000:]
y5_train = y5[10000:]

print(len(L), len(L_valid), len(L_train))

33402 10000 23402
33402 10000 23402


In [444]:


image_size = 32 # 32 x 32 pixel
num_channels = 1 # grayscale
num_labels = 11 # 0-9 + 10 (for missing value)
num_digits = 6 #max 5 digit for housenumber

import numpy as np

def reformat(dataset, label0, label1, label2, label3, label4, label5):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  print((label0[:10]))
  print((label1[:10]))
  print((label2[:10]))
  print((label3[:10]))
  print(np.asarray(label3[:10]))
  label0 = (np.arange(num_digits) == np.asarray(label0)[:,None]).astype(np.float32)
  label1 = (np.arange(num_labels) == np.asarray(label1)[:,None]).astype(np.float32)
  label2 = (np.arange(num_labels) == np.asarray(label2)[:,None]).astype(np.float32)
  label3 = (np.arange(num_labels) == np.asarray(label3)[:,None]).astype(np.float32)
  label4 = (np.arange(num_labels) == np.asarray(label4)[:,None]).astype(np.float32)
  label5 = (np.arange(num_labels) == np.asarray(label5)[:,None]).astype(np.float32)

  print(np.asarray(label3[:10]))
  return dataset, label0, label1, label2, label3, label4, label5
train_dataset, train_label0, train_label1, train_label2, train_label3, train_label4, train_label5 = reformat(np.asarray(X_train), L_train, y1_train, y2_train, y3_train, y4_train, y5_train)
valid_dataset, valid_label0, valid_label1, valid_label2, valid_label3, valid_label4, valid_label5 = reformat(np.asarray(X_valid), L_valid, y1_valid, y2_valid, y3_valid, y4_valid, y5_valid)
#test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_label0.shape, train_label1.shape)
print('Validation set', valid_dataset.shape, valid_label0.shape, valid_label1.shape)
#print('Test set', test_dataset.shape, test_labels.shape)

[2, 3, 2, 2, 4, 2, 2, 2, 1, 2]
[4, 2, 5, 8, 1, 4, 1, 9, 6, 1]
[1, 1, 9, 8, 10, 2, 6, 2, 10, 3]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10 10 10 10 10 10 10 10 10 10]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
[2, 2, 2, 2, 2, 2, 2, 3, 3, 2]
[1, 2, 2, 9, 3, 3, 2, 7, 1, 1]
[9, 3, 5, 3, 1, 3, 8, 4, 2, 6]
[10, 10, 10, 10, 10, 10, 10, 4, 8, 10]
[10 10 10 10 10 10 10  4  8 10]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.

# Import Modules 

In [264]:
import tensorflow as tf
import numpy as np

# Neural Network Configurations

In [450]:
batch_size = 256

# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

#Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36        # There are 36 of these filters.

#Convolutional Layer 2.
filter_size3 = 5          # Convolution filters are 5 x 5 pixels.
num_filters3 = 64        # There are 64 of these filters.

# Fully-connected layer.
fc_size = 1024         # Number of neurons in fully-connected hidden layer

drop_rate = 0.9

# Create graph

### Declare variables to initialize

In [469]:
graph = tf.Graph()

with graph.as_default():
#with tf.device('/gpu:0'):
  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels0 = tf.placeholder(tf.float32, shape=(batch_size, num_digits))
  tf_train_labels1 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels2 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels3 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels4 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels5 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
 # tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  #w1 = tf.get_variable("w1", shape=[filter_size1, filter_size1, num_channels, num_filters1], initializer=tf.contrib.layers.xavier_initializer()) 
  #w1 = tf.Variable(tf.truncated_normal([filter_size1, filter_size1, num_channels, num_filters1], stddev=0.1)) # 5, 5, 1, 16
  w1cv1 = tf.Variable(tf.truncated_normal([filter_size1,filter_size1, num_channels, num_filters1], stddev=np.sqrt(1/(filter_size1*filter_size1*num_channels)))) # 5, 5, 1, 16
  b1cv1 = tf.Variable(tf.zeros([num_filters1])) #16

  w1cv2 = tf.Variable(tf.truncated_normal([filter_size1,filter_size1, num_filters1, num_filters1], stddev=np.sqrt(1/(filter_size1*filter_size1*num_filters1)))) # 5, 5, 16, 16
  b1cv2 = tf.Variable(tf.constant(0.1, shape=[num_filters1])) #16
    
  w1cv3 = tf.Variable(tf.truncated_normal([filter_size1,filter_size1, num_filters1, num_filters1], stddev=np.sqrt(1/(filter_size1*filter_size1*num_filters1)))) # 5, 5, 16, 16
  b1cv3 = tf.Variable(tf.constant(0.1, shape=[num_filters1])) #16

#pool 16

  w2cv1 = tf.Variable(tf.truncated_normal([filter_size2,filter_size2, num_filters1, num_filters2], stddev=np.sqrt(1/(filter_size1*filter_size2*num_filters1)))) # 5, 5, 16, 36
  b2cv1 = tf.Variable(tf.constant(0.1, shape=[num_filters2])) #36

  w2cv2 = tf.Variable(tf.truncated_normal([filter_size2,filter_size2, num_filters2, num_filters2], stddev=np.sqrt(1/(filter_size1*filter_size2*num_filters2)))) # 5, 5, 36, 36
  b2cv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2])) #36

  w2cv3 = tf.Variable(tf.truncated_normal([filter_size2,filter_size2, num_filters2, num_filters2], stddev=np.sqrt(1/(filter_size1*filter_size2*num_filters2)))) # 5, 5, 36, 36
  b2cv3 = tf.Variable(tf.constant(0.1, shape=[num_filters2])) #36
  
#pool 16

  w3cv1 = tf.Variable(tf.truncated_normal([filter_size3,filter_size3, num_filters2, num_filters3], stddev=np.sqrt(1/(filter_size3*filter_size3*num_filters2)))) # 5, 5, 16, 64
  b3cv1 = tf.Variable(tf.constant(0.1, shape=[num_filters3])) #64

  w3cv2 = tf.Variable(tf.truncated_normal([filter_size3,filter_size3, num_filters3, num_filters3], stddev=np.sqrt(1/(filter_size3*filter_size3*num_filters3)))) # 5, 5, 64, 64
  b3cv2 = tf.Variable(tf.constant(0.1, shape=[num_filters3])) #64
  
  w3cv3 = tf.Variable(tf.truncated_normal([filter_size3,filter_size3, num_filters3, num_filters3], stddev=np.sqrt(1/(filter_size3*filter_size3*num_filters3)))) # 5, 5, 64, 64
  b3cv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3])) #64
    
 #pool 8

  #wfcin = image_size // 2*2*2 * image_size // 2*2*2 *num_filters3 #(3 max pool @ stride 2)
  wfc = tf.Variable(tf.truncated_normal([fc_size, batch_size], stddev=np.sqrt(2/(fc_size + batch_size)))) #4*4*64=1024,256 
  bfc = tf.Variable(tf.constant(0.1, shape=[batch_size])) #256
    
  w0fc = tf.Variable(tf.truncated_normal([batch_size, num_digits], stddev=np.sqrt(2/(batch_size + num_digits)))) #256, 6
  b0fc = tf.Variable(tf.constant(0.1, shape=[num_digits])) #6
    
  w1fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b1fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w2fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b2fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w3fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b3fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w4fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b4fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w5fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b5fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11

#init = tf.contrib.layers.xaviar_initializer()
#h = tf.layers.dense(inputs=x, units=image_size, activation=tf.nn.relu, kernel_initializer=init)
#y_pred = tf.layers.dense(inputs=h, units = 1000, kernel_initializer=init)

# Model.
  def model(data):
    ######
    # CNN
    ######
    # x = 32x32 pixel * 1 channel* 256 batch samples
    # w = 5x5 filter * 1 channel * 16 filters/depth
    # b1= 16 depth
    # conv/hidden = 32x32 pixel * 16 depth * 256 batch samples  (32-5+(2*2)[padding to be same output])/1[stride]+1
    conv1a = tf.nn.conv2d(data, w1cv1, [1, 1, 1, 1], padding='SAME')
    hidden1a =tf.maximum((conv1a + b1cv1), 0.01 * (conv1a + b1cv1)) # tf.nn.relu(conv1 + b1) #
    
    # hidden = 32x32 pixel * 16 channel* 256 batch samples
    # w = 5x5 filter * 16 channel * 16 filters/depth
    # b = 16 depth
    # conv/hidden = 32x32 pixel * 16 depth * 256 batch samples  (32-5+(2*2)[padding to be same output])/1[stride]+1
    conv2a = tf.nn.conv2d(hidden1a, w1cv2, [1, 1, 1, 1], padding='SAME')
    hidden2a =tf.maximum((conv2a + b1cv2), 0.01 * (conv2a + b1cv2)) #tf.nn.relu(conv2 + b2)# 
    
    conv3a = tf.nn.conv2d(hidden2a, w1cv3, [1, 1, 1, 1], padding='SAME')
    hidden3a =tf.maximum((conv3a + b1cv3), 0.01 * (conv3a + b1cv3)) #tf.nn.relu(conv3 + b3)# 
    
    # pool = 16x16 pixel * 16 depth * 256 batch samples
    pool1a = tf.nn.max_pool(hidden3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    ###
    
    # pool = 8x8 pixel * 36 channel* 256 batch samples
    # w = 5x5 filter * 36 channel * 36 filters/depth
    # b = 36 depth
    # conv/hidden = 8x8 pixel * 36 depth * 256 batch samples  (8-5+(2*2)[padding to be same output])/1[stride]+1
    conv1b = tf.nn.conv2d(pool1a, w2cv1, [1, 1, 1, 1], padding='SAME')
    hidden1b = tf.maximum((conv1b + b2cv1), 0.01 * (conv1b + b2cv1)) #tf.nn.relu(conv1b + b1b)#
    
    # hidden = 8x8 pixel * 36 depth * 256 batch samples 
    # w = 5x5 filter * 36 channel * 36 filters/depth
    # b = 64 depth
    # conv/hidden = 8x8 pixel * 64 depth * 256 batch samples  (8-5+(2*2)[padding to be same output])/1[stride]+1
    conv2b = tf.nn.conv2d(hidden1b, w2cv2, [1, 1, 1, 1], padding='SAME')
    hidden2b =tf.maximum((conv2b + b2cv2), 0.01 * (conv2b + b2cv2)) #tf.nn.relu(conv2b + b2b)# 
    #print(w.get_shape())
    #print(b.get_shape())
    #print(hidden.get_shape())
    
    conv3b = tf.nn.conv2d(hidden2b, w2cv3, [1, 1, 1, 1], padding='SAME')
    hidden3b =tf.maximum((conv3b + b2cv3), 0.01 * (conv3b + b2cv3)) #tf.nn.relu(conv3 + b3)# 
    
    # pool = 4x4 pixel * 36 depth * 256 batch samples
    pool1b = tf.nn.max_pool(hidden3b, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #print(pool.get_shape())
    
    ###
    
    # pool = 4x4 pixel * 36 channel* 256 batch samples
    # w = 5x5 filter * 36 channel * 64 filters/depth
    # b = 64 depth
    # conv/hidden = 4x4 pixel * 64 depth * 256 batch samples  (4-5+(2*2)[padding to be same output])/1[stride]+1
    conv1c = tf.nn.conv2d(pool1b, w3cv1, [1, 1, 1, 1], padding='SAME')
    hidden1c = tf.maximum((conv1c + b3cv1), 0.01 * (conv1c + b3cv1)) #tf.nn.relu(conv1b + b1b)#
    
    # hidden = 4x4 pixel * 64 depth * 256 batch samples 
    # w = 5x5 filter * 64 channel * 64 filters/depth
    # b = 64 depth
    # conv/hidden = 4x4 pixel * 64 depth * 256 batch samples  (4-5+(2*2)[padding to be same output])/1[stride]+1
    conv2c = tf.nn.conv2d(hidden1c, w3cv2, [1, 1, 1, 1], padding='SAME')
    hidden2c =tf.maximum((conv2c + b3cv2), 0.01 * (conv2c + b3cv2)) #tf.nn.relu(conv2b + b2b)# 
    #print(w.get_shape())
    #print(b.get_shape())
    #print(hidden.get_shape())
    
    conv3c = tf.nn.conv2d(hidden2c, w3cv3, [1, 1, 1, 1], padding='SAME')
    hidden3c =tf.maximum((conv3c + b3cv3), 0.01 * (conv3c + b3cv3)) #tf.nn.relu(conv3 + b3)# 
    
    # pool = 8x8 pixel * 64 depth * 256 batch samples
    pool1c = tf.nn.max_pool(hidden3c, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #print(pool.get_shape())
        
    #####
    # FC Layers
    #####
    shape = pool1c.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(pool1c, [shape[0], shape[1] * shape[2] * shape[3]]) #[256, 4, 4, 64]
    print(reshape) #256 * 4*4*64=1024
    print(wfc.get_shape())
    print(bfc.get_shape())
    # w = 1024 depth * 1024 FC hidden node layer
    # b = 1024 FC hidden node layers
    y = tf.maximum((tf.matmul(reshape, wfc) + bfc), 0.01 * tf.matmul(reshape, wfc) + bfc) #tf.nn.relu(tf.matmul(reshape, w3) + b3)
    y = tf.nn.dropout(y, drop_rate)  # Dropout
    

    return y #tf.matmul(hidden, layer4_weights) + layer4_biases

  # Training computation.
  # w4 = 256 batch sample * 6 classifier
  # b4 = 6 classifier
  # 256 batch sample x 6 classifier
  logits =  tf.matmul(model(tf_train_dataset), w0fc) + b0fc
    
  # w4b = 256 batch sample * 11 classifier
  # b4b = 11 classifier
  # 256 batch sample x 11 classifier
  logits1 = tf.matmul(model(tf_train_dataset), w1fc) + b1fc
  logits2 = tf.matmul(model(tf_train_dataset), w2fc) + b2fc
  logits3 = tf.matmul(model(tf_train_dataset), w3fc) + b3fc
  logits4 = tf.matmul(model(tf_train_dataset), w4fc) + b4fc
  logits5 = tf.matmul(model(tf_train_dataset), w5fc) + b5fc
    
  loss0 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels0, logits=logits)
  loss1 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels1, logits=logits1)
  loss2 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels2, logits=logits2)
  loss3 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels3, logits=logits3)
  loss4 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels4, logits=logits4)
  loss5 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels5, logits=logits5)
 # loss =  tf.reduce_mean(loss0 + loss1 + loss2 + loss3 + loss4 + loss5)

  loss= tf.reduce_mean(loss0) + tf.reduce_mean(loss1) + tf.reduce_mean(loss2) + tf.reduce_mean(loss3) + tf.reduce_mean(loss4) + tf.reduce_mean(loss5)
    
  weight = wfc
  weight1 = w1fc
  # Optimizer.
 # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0) #, trainable=False)  # count the number of steps taken.
 # learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
 # learning_rate = tf.train.exponential_decay(0.1, global_step, 200, 0.7) #, staircase=True)
 # optimizer = tf.train.GradientDescentOptimizer(1e-6).minimize(loss) #, global_step=global_step)
  optimizer =tf.train.AdamOptimizer(0.001).minimize(loss) #, global_step=global_step) 

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits1)
 # valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))


[256, 4, 4, 64]
Tensor("Reshape:0", shape=(256, 1024), dtype=float32)
(1024, 256)
(256,)
[256, 4, 4, 64]
Tensor("Reshape_1:0", shape=(256, 1024), dtype=float32)
(1024, 256)
(256,)
[256, 4, 4, 64]
Tensor("Reshape_2:0", shape=(256, 1024), dtype=float32)
(1024, 256)
(256,)
[256, 4, 4, 64]
Tensor("Reshape_3:0", shape=(256, 1024), dtype=float32)
(1024, 256)
(256,)
[256, 4, 4, 64]
Tensor("Reshape_4:0", shape=(256, 1024), dtype=float32)
(1024, 256)
(256,)
[256, 4, 4, 64]
Tensor("Reshape_5:0", shape=(256, 1024), dtype=float32)
(1024, 256)
(256,)


### Run graph session

In [470]:
import time

t1 = time.time()

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#def eval_accuracy(eval_l_preds, eval_preds, l_labels, labels, masks):
#    concatted = np.concatenate((np.reshape((eval_l_preds == l_labels), [-1, 1]), 
#                                (eval_preds * masks) == labels), axis=1)
#    return 100.0 * (np.sum([np.all(row) for row in concatted])) / len(labels)
#
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # sess.run(tf.global_variables_initializer())
  print('Initialized')
    
#  for step in range(num_steps):
 #   offset = (step * batch_size) % ( np.asarray(L_train).shape[0] - batch_size)
   # batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
  #  batch_labels0 =  np.asarray(L_train)[offset:(offset + batch_size)]  
  #  feed_dict = {#tf_train_dataset : batch_data
   #               tf_train_labels0 : batch_labels0}
  for step in range(num_steps):
    offset = (step * batch_size) % (np.asarray(y1_train).shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :] #, :, :]
    
    #normalize data to mean 0 and unit variance
    batch_data = (batch_data - np.mean(batch_data))/np.std(batch_data)
    #batch_data = np.reshape(batch_data,(batch_size,image_size*image_size))
   # print(batch_data.shape)
    batch_labels0 =  np.asarray(train_label0)[offset:(offset + batch_size), :]
    batch_labels1 =  np.asarray(train_label1)[offset:(offset + batch_size), :]
    batch_labels2 =  np.asarray(train_label2)[offset:(offset + batch_size), :]
    batch_labels3 =  np.asarray(train_label3)[offset:(offset + batch_size), :]
    batch_labels4 =  np.asarray(train_label4)[offset:(offset + batch_size), :]
    batch_labels5 =  np.asarray(train_label5)[offset:(offset + batch_size), :]    
   # print(batch_labels1.shape)
    feed_dict = {tf_train_dataset : batch_data
                , tf_train_labels0 : batch_labels0
                , tf_train_labels1 : batch_labels1
                , tf_train_labels2 : batch_labels2
                , tf_train_labels3 : batch_labels3
                , tf_train_labels4 : batch_labels4
                , tf_train_labels5 : batch_labels5
                }
    _, l, predictions, w, w1 = session.run([optimizer, loss, train_prediction, weight, weight1], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels1))
      print("w1 mean: {} std: {}".format(np.mean(w), np.std(w)))
      print("wy1 mean: {} std: {}".format(np.mean(w1), np.std(w1)))
      #print("label: ", batch_labels1)
    #  print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), L_valid))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

t2 = time.time()

print("Time: %0.2fs" % (t2 - t1))

Initialized
Minibatch loss at step 0: 14.929310
Minibatch accuracy: 16.4%
w1 mean: 7.922627264633775e-05 std: 0.034734081476926804
wy1 mean: 0.001830380060710013 std: 0.07737302780151367
Minibatch loss at step 50: 6.942143
Minibatch accuracy: 23.8%
w1 mean: -0.0010970375733450055 std: 0.035305578261613846
wy1 mean: -0.0013414585264399648 std: 0.07756786793470383
Minibatch loss at step 100: 6.423608
Minibatch accuracy: 29.7%
w1 mean: -0.0011747158132493496 std: 0.035367149859666824
wy1 mean: -0.002143967431038618 std: 0.07772453874349594
Minibatch loss at step 150: 6.607699
Minibatch accuracy: 27.3%
w1 mean: -0.0013206790899857879 std: 0.03555382788181305
wy1 mean: -0.002545771887525916 std: 0.07770256698131561
Minibatch loss at step 200: 6.242787
Minibatch accuracy: 27.0%
w1 mean: -0.0015731240855529904 std: 0.03591102361679077
wy1 mean: -0.0034138737246394157 std: 0.07788414508104324
Minibatch loss at step 250: 6.083212
Minibatch accuracy: 27.3%
w1 mean: -0.0018892630469053984 std: 0.